In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

### Lendo dataset a partir do arquivo excel extraido direto do site do CEPEA

In [122]:
df_read = pd.read_excel(open('data/CEPEA_20210315181759.xlsx', 'rb'), sheet_name='Plan 1')

In [123]:
df = df_read.iloc[3:, :3]
df.columns = ['data', 'preco_br', 'preco_us']

### Transformando a coluna data em formato americano

In [124]:
data = df['data']
ano = [a.split('/')[-1] for a in data]
dia = [a.split('/')[0] for a in data]
mes = [a.split('/')[1] for a in data]

In [125]:
arr_list = [ano, mes, dia]
new_data = np.apply_along_axis('-'.join, 0, arr_list)
df['data'] = new_data

In [126]:
df['data'] = pd.to_datetime(df['data'])
df = df.set_index('data')
df['2020-01']

,preco_br,preco_us
data,,
2020-01-02,82.92,20.61
2020-01-03,83.01,20.48
2020-01-06,82.83,20.35
2020-01-07,82.81,20.4
2020-01-08,82.95,20.49
2020-01-09,82.83,20.27
2020-01-10,82.83,20.3
2020-01-13,83.59,20.2
2020-01-14,83.78,20.29


### Feature Engineering

In [134]:
df2 = df[['preco_br']]

df2.loc[:, 'yesterday'] = df2.loc[:,'preco_br'].shift()
df2.loc[:, 'yesterday_dif'] = df2.loc[:, 'yesterday'].diff()

df2.loc[:, 'yesterday_1'] = df2.loc[:, 'yesterday'].shift()
# df2.loc[:, 'yesterday_1_dif'] = df2.loc[:, 'yesterday_1'].diff()

# df2.loc[:, 'last_week'] = df2.loc[:, 'yesterday'].shift(7)
# df2.loc[:, 'last_week_diff'] = df2.loc[:, 'last_week'].diff()


# df2.loc[:, 'last_year'] = df2.loc[:, 'yesterday'].shift(365)
# df2.loc[:, 'last_year_diff'] = df2.loc[:, 'last_year'].diff()
# df2.loc[:, 'last_year_diff'] = df2['yesterday'].sub(df2['last_year'], axis = 0) 

# df2.loc[:, 'last_month'] = df2.loc[:, 'yesterday'].shift(30)
# df2.loc[:, 'last_month_diff'] = df2.loc[:, 'last_month'].diff()

df2 = df2.dropna()
df2

,preco_br,yesterday,yesterday_dif,yesterday_1
data,,,,
1997-07-31,18.17,17.97,-0.07,18.04
1997-08-01,18.1,18.17,0.2,17.97
1997-08-04,18.23,18.1,-0.07,18.17
1997-08-05,18.16,18.23,0.13,18.1
1997-08-06,18.15,18.16,-0.07,18.23
...,...,...,...,...
2021-03-08,169.58,166.4,1.55,164.85
2021-03-09,171.42,169.58,3.18,166.4
2021-03-10,169.31,171.42,1.84,169.58


### Testing Cross-Validation on Time Series

In [135]:
year_list = df2.index.year.unique().tolist()
splits = {'train': [], 'test': []}

for idx, yr in enumerate(year_list[:-1]):
    train_yr = year_list[:idx+1]
    test_yr = [year_list[idx+1]]
    print('TRAIN: ', train_yr, 'TEST: ',test_yr)
    
    splits['train'].append(df2.loc[df2.index.year.isin(train_yr), :])
    splits['test'].append(df2.loc[df2.index.year.isin(test_yr), :])

TRAIN:  [1997] TEST:  [1998]
TRAIN:  [1997, 1998] TEST:  [1999]
TRAIN:  [1997, 1998, 1999] TEST:  [2000]
TRAIN:  [1997, 1998, 1999, 2000] TEST:  [2001]
TRAIN:  [1997, 1998, 1999, 2000, 2001] TEST:  [2002]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002] TEST:  [2003]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003] TEST:  [2004]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004] TEST:  [2005]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005] TEST:  [2006]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006] TEST:  [2007]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007] TEST:  [2008]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008] TEST:  [2009]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009] TEST:  [2010]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010] TEST:  [2011]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 200

In [136]:
model = LinearRegression()

for i, yr in enumerate(year_list[:-1]):
    X_train = splits['train'][i].drop(['preco_br'], axis = 1).values # X
    y_train = splits['train'][i]['preco_br'].values # Y
    
    X_test =  splits['test'][i].drop(['preco_br'], axis = 1).values
    y_test = splits['test'][i]['preco_br'].values
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    
    r2 = metrics.r2_score(y_test, pred)
    print(' {} >>> R²: {}'.format(yr, r2))

 1997 >>> R²: 0.9740777197233937
 1998 >>> R²: 0.9836594031503909
 1999 >>> R²: 0.9732634111460481
 2000 >>> R²: 0.9958595220900075
 2001 >>> R²: 0.9970134808018746
 2002 >>> R²: 0.9890961631794772
 2003 >>> R²: 0.9924985821907611
 2004 >>> R²: 0.9553219647781999
 2005 >>> R²: 0.9885636999122454
 2006 >>> R²: 0.9967113509084556
 2007 >>> R²: 0.9441014452158959
 2008 >>> R²: 0.9718163954602057
 2009 >>> R²: 0.9958352592195613
 2010 >>> R²: 0.9617216551758261
 2011 >>> R²: 0.9983765161945022
 2012 >>> R²: 0.9887331944340432
 2013 >>> R²: 0.9735393189920807
 2014 >>> R²: 0.9927170737127046
 2015 >>> R²: 0.9814225728538758
 2016 >>> R²: 0.9680219281641389
 2017 >>> R²: 0.9890747791474694
 2018 >>> R²: 0.9890292183318132
 2019 >>> R²: 0.9984986382225891
 2020 >>> R²: 0.82467041644329


In [137]:
df2.to_csv('data/final/soja_cepea_final.csv')